In [1]:
import numpy as np
import helper_functions as hf
from scipy.stats import kurtosis, skew
from biosppy.signals.emg import emg


In [2]:
X_train_eeg1, _ = hf.read_csv_to_matrix("task5/input/train_eeg1.csv", "Id")
X_train_eeg2, _ = hf.read_csv_to_matrix("task5/input/train_eeg2.csv", "Id")
X_train_emg, _ = hf.read_csv_to_matrix("task5/input/train_emg.csv", "Id")
X_test_eeg1, _ = hf.read_csv_to_matrix("task5/input/train_eeg1.csv", "Id")
X_test_eeg2, _ = hf.read_csv_to_matrix("task5/input/train_eeg2.csv", "Id")
X_test_emg, _ = hf.read_csv_to_matrix("task5/input/train_emg.csv", "Id")
y_train, _ = hf.read_csv_to_matrix("task5/input/train_labels.csv", "Id")
_, test_index = hf.read_csv_to_matrix("task5/input/sample.csv", "Id")
y_train = np.squeeze(y_train)


In [3]:
def eeg_feature_extraction(x):
    x_new = []
    for i in range(x.shape[0]):
        x_new.append([np.mean(x[i]), np.std(x[i]), kurtosis(x[i]), skew(x[i])])
    return np.asarray(x_new)


In [4]:
X_train_eeg1_new = eeg_feature_extraction(X_train_eeg1)


In [27]:
X_test_emg_new = []
ts = []
filtered = []
onsets = []
for i in range(X_train_emg.shape[0]):
    ts[i], filtered[i], onsets[i] = emg(X_train_emg[i])


In [21]:
from biosppy import tools as st


def emg(signal=None, sampling_rate=1000, show=True):
    # check inputs
    if signal is None:
        raise TypeError("Please specify an input signal.")

    # ensure numpy
    signal = np.array(signal)

    sampling_rate = float(sampling_rate)

    # filter signal
    filtered, _, _ = st.filter_signal(signal=signal,
                                      ftype='butter',
                                      band='highpass',
                                      order=4,
                                      frequency=100,
                                      sampling_rate=sampling_rate)

    # find onsets
    onsets, = find_onsets(signal=filtered, sampling_rate=sampling_rate)

    # get time vectors
    length = len(signal)
    T = (length - 1) / sampling_rate
    ts = np.linspace(0, T, length, endpoint=False)

    # plot
    if show:
        plotting.plot_emg(ts=ts,
                          sampling_rate=1000.,
                          raw=signal,
                          filtered=filtered,
                          processed=None,
                          onsets=onsets,
                          path=None,
                          show=True)

    # output
    args = (ts, filtered, onsets)
    names = ('ts', 'filtered', 'onsets')

    return utils.ReturnTuple(args, names)


ValueError: Digital filter critical frequencies must be 0 < Wn < 1

In [22]:
a = emg(X_train_emg[0], sampling_rate=128, show=False)


ValueError: Digital filter critical frequencies must be 0 < Wn < 1

In [9]:
a['onsets']

array([], dtype=int64)

In [6]:
from biosppy import tools as st
frequency = st._norm_freq(frequency=100, sampling_rate=1280)


In [7]:
frequency

0.15625